### Vectorize Data Sets

In [7]:
from numba import jit, autojit # accelerate for loops
import numpy as np
import numpy.random as npr
import re # regular expressions
import glob
import pandas as pd

In [8]:
###--- Methods ---###

#### bootstrapping: resampling (virtual example)

nb = 100  # how many samples ?
ln = 3000 # how many words for each sample ? (3000 ~ length of each text)

def btp_ori(tot,nb,ln):
    if len(tot) == 0: return np.empty(0)
    else:
        idx = npr.randint(0, len(tot), (nb,ln)) # randomly pick indices from the 'at' array, 'nb' rows
        return np.array(tot[idx])

#### extract function words
# function word list: Bei Yu 2012: Function Words for Chinese Authorship Attribution
fw1 = ["的","是","不","了","在","有","這","為","地","也","得","就","那","以","著","之","可","于"
       "麼","而","然","没","於","還","只","無","又","如","但","其","此","與","把","全","被","卻"]

# difficult word list: 司法院裁判書類通俗化小組
fw2 = ["殆","似","難認無過失","難認有理由","尚難謂為於法無違","非無理由","非有理由","尚非無疑","尚非不可能",
       "可徵","徵諸","洵堪","迭於","係經","係指","洵屬","伊","渠","致","爰","遽認","矧","略以","按","第按","是","茲",
       "揆諸","皆無該條適用","即","無庸","縱",
       "上開","足資","可稽","堪以認定","業據","云云","等語","之","次查","復查","再查","系爭","固非無見","旋","惟查","均係"]
fwt = fw1+fw2

@jit
def ext_fw(txt,fw):
    # txt = input texts; fw = function word list
    j = np.empty(0)
    for i in fw:
        l = re.findall(i,txt)
        j = np.concatenate((j,np.array(l)))
    return j

In [9]:
#### generate csv files
global file_idx
file_idx = 0
def fw_csv(path):
    list_of_files = glob.glob(path)
    
    at = np.empty(0)
    for file_n in list_of_files:
        print file_n
        with open(file_n,'r') as myfile:         
            sl = myfile.read().replace('\n','')   # returns a string
            sa = np.array(re.findall(r'\S+', sl)) # returns an array of the above string
            
            # all texts as one
            at = np.concatenate((at,sa))
        
        btp = btp_ori(at,nb,ln) # virtual examples: # = nb, length = ln

        for i in range(btp.shape[0]):
            bts = " ".join(btp[i])
            fwr = ext_fw(bts,fw1+fw2)

            # frequency table
            fwr_str = "\t".join(fwr).split("\t")
            fwr_tb  = pd.Series(fwr_str).value_counts()

            # store in dictionary
            dict1 = {}
            dict1.update(fwr_tb)
            dict1.update({"NAME": "%s" %f})
            dict1.update({"file_n": file_idx})
            tp.append(dict1)
        global file_idx
        file_idx+=1
       
    return tp
    

<ipython-input-9-068a1c1b8991>:33: SyntaxWarning: name 'file_idx' is used prior to global declaration
  global file_idx


In [10]:
#### read corpus
list_of_jud   = ["翁岳生","城仲模","林永謀","王和雄","余雪明","廖義男","曾有田","楊仁壽",
                 "彭鳳至","賴英照","謝在全","徐璧湖","林子儀","許宗力","許玉秀","林錫堯",
                 "池啟明","李震山","蔡清遊","賴浩敏","蘇永欽","黃茂榮","陳新民","陳春生","陳敏","葉百修"]

tp = [] # an empty list
file_idx = 0
for f in list_of_jud:
    # for each author ...
    path = './segmented_txt_2/%s/*.txt' %f 
    rs   = fw_csv(path)
    
df = pd.DataFrame(rs) 
# output csv
df.to_csv("./vectorized.csv", encoding="utf-8")

./segmented_txt_2/翁岳生/翁岳生455_seg.txt
./segmented_txt_2/城仲模/城仲模604_seg.txt
./segmented_txt_2/城仲模/城仲模603_seg.txt
./segmented_txt_2/城仲模/城仲模391_seg.txt
./segmented_txt_2/林永謀/林永謀374_seg.txt
./segmented_txt_2/林永謀/林永謀588_seg.txt
./segmented_txt_2/林永謀/林永謀372_seg.txt
./segmented_txt_2/林永謀/林永謀393_seg.txt
./segmented_txt_2/林永謀/林永謀370_seg.txt
./segmented_txt_2/林永謀/林永謀442_seg.txt
./segmented_txt_2/林永謀/林永謀590_seg.txt
./segmented_txt_2/林永謀/林永謀461_seg.txt
./segmented_txt_2/林永謀/林永謀413_seg.txt
./segmented_txt_2/林永謀/林永謀467_seg.txt
./segmented_txt_2/林永謀/林永謀392_seg.txt
./segmented_txt_2/林永謀/林永謀376_seg.txt
./segmented_txt_2/林永謀/林永謀384_seg.txt
./segmented_txt_2/林永謀/林永謀436_seg.txt
./segmented_txt_2/林永謀/林永謀403_seg.txt
./segmented_txt_2/林永謀/林永謀499_seg.txt
./segmented_txt_2/王和雄/王和雄392_seg.txt
./segmented_txt_2/王和雄/王和雄550_seg.txt
./segmented_txt_2/王和雄/王和雄490_seg.txt
./segmented_txt_2/余雪明/余雪明596_seg.txt
./segmented_txt_2/余雪明/余雪明603_seg.txt
./segmented_txt_2/余雪明/余雪明613_seg.txt
./segmented_txt_2/余雪明/余雪明632_seg.txt
.

---
### Examine results:

In [11]:
import pandas as pd
test = pd.read_csv("./vectorized.csv", encoding="utf-8")
test

,Unnamed: 0,NAME,file_n,上開,不,之,也,了,云云,以,...,致,與,茲,著,被,足資,這,遽認,還,那
0,0,翁岳生,0,1,62,380,NaN,NaN,NaN,50,...,3,27,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN
1,1,翁岳生,0,2,44,416,NaN,NaN,NaN,40,...,4,39,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
2,2,翁岳生,0,NaN,46,364,NaN,NaN,NaN,39,...,4,32,NaN,2,1,NaN,NaN,NaN,NaN,NaN
3,3,翁岳生,0,4,53,372,NaN,NaN,NaN,40,...,2,54,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN
4,4,翁岳生,0,1,36,360,NaN,NaN,NaN,43,...,8,41,NaN,4,1,NaN,NaN,NaN,NaN,NaN
5,5,翁岳生,0,2,48,356,NaN,NaN,NaN,48,...,4,38,NaN,2,2,NaN,NaN,NaN,NaN,NaN
6,6,翁岳生,0,1,58,348,NaN,NaN,NaN,45,...,2,38,NaN,3,1,NaN,NaN,NaN,NaN,NaN
7,7,翁岳生,0,5,34,396,NaN,NaN,NaN,49,...,5,39,NaN,3,1,NaN,NaN,NaN,NaN,NaN
8,8,翁岳生,0,1,46,348,NaN,NaN,NaN,48,...,4,36,NaN,5,1,NaN,NaN,NaN,NaN,NaN
9,9,翁岳生,0,1,46,362,NaN,NaN,NaN,42,...,3,39,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


---
### Generate testing data from judicial Yuan (cf. parser.ipynb)

In [ ]:
tp = [] # an empty list
path = './test_d/*.txt'
rs   = fw_csv(path)
    
df = pd.DataFrame(rs) 
# output csv
df.to_csv("./test_ds.csv", encoding="utf-8")